# Data Mining of Human Skin Microbiome from EBI-Metagenomics Portal

_Matin Nuhamunada_<sup>1*</sup>, _Gregorius Altius Pratama_<sup>1</sup>, _Setianing Wikanthi_<sup>2</sup>, and _Mohamad Khoirul Anam_<sup>1</sup>

<sup>1</sup>Department of Tropical Biology, Universitas Gadjah Mada;   
Jl. Teknika Selatan, Sekip Utara, Bulaksumur, Yogyakarta, Indonesia, 55281;   

<sup>2</sup>Department of Agricultural Microbiology, Universitas Gadjah Mada;  

*Correspondence: [matin_nuhamunada@ugm.ac.id](mailto:matin_nuhamunada@mail.ugm.ac.id)  
[mohamad.khoirul.anam@mail.ugm.ac.id](mailto:mohamad.khoirul.anam@mail.ugm.ac.id)  
[gregorius.altius.p@mail.ugm.ac.id](mailto:gregorius.altius.p@mail.ugm.ac.id)  
[setianingwikanthi@mail.ugm.ac.id](mailto:setianingwikanthi@mail.ugm.ac.id)

---
## Abstract
Human skin microbiome is unique to individuals in regards to many aspects, including behaviour, environment, and perhaps maybe genes. To understand more about the distribution of human skin microbiome across the globe, we compare several skin microbiome study available in the EBI-Metagenomic Portal. Study data was acquired using EBI-Metagenome API, and sample data was selected based on sex, location, and bodysite. The biological observation matrix from the analysis result of the selected samples were compared using MEGAN. 

### Keywords
Human Skin, Microbiome, EBI-Metagenome


## Import Python Modules
We use python 3 script using ``pandas``, ``jsonapi_client``, ``pycurl``, to mine the data from EBI-metagenomic portal  [[1]](#ref1).

In [112]:
from pandas import DataFrame
import pandas as pd

try:
    from urllib import urlencode
except ImportError:
    from urllib.parse import urlencode

In [113]:
from jsonapi_client import Session, Filter

API_BASE = 'https://www.ebi.ac.uk/metagenomics/api/latest/'

In [114]:
import pycurl
import os, sys
from tqdm import tqdm_notebook
import numpy as np
from time import sleep

## Load Functions

In [182]:
def get_metadata(metadata, key):
    import html
    for m in metadata:
        if m['key'].lower() == key.lower():
            value = m['value']
            unit = html.unescape(m['unit']) if m['unit'] else ""
            return "{value} {unit}".format(value=value, unit=unit)
    return None

def get_study(term, lineage, biome, filename):
    if not os.path.isfile(filename):
        with open(filename, 'wb') as f:
            c = pycurl.Curl()
            c.setopt(c.URL, 'https://www.ebi.ac.uk/metagenomics/projects/doExportDetails?searchTerm='+term+'&includingChildren=true&biomeLineage=root%3A'+lineage+'%3A'+biome+'&search=Search')
            c.setopt(c.WRITEDATA, f)
            c.perform()
            c.close()
    return filename

def get_analysis_result(run, extension):
    API_BASE_RUN = 'https://www.ebi.ac.uk/metagenomics/api/latest/runs'
    with Session(API_BASE_RUN) as s:
        study = s.get(run,'analysis').resource
        for i in study.downloads:
            if extension in i.file_format['name']:
                i.url
    return i.url

## Get Study
We search the EBI Metagenomic database on human skin microbiome study in the host-associated biome with 'skin' as search term. The study list can be found on this link: https://www.ebi.ac.uk/metagenomics/projects/doExportDetails?searchTerm=skin&includingChildren=true&biomeLineage=root%3AHost-associated%3AHuman&search=Search

In [116]:
#Search Study
term = 'skin'
lineage = 'Host-associated'
biome = 'Human'
file_study = '01_Study_'+term+'+'+biome+'+'+lineage+'_raw.csv'

In [117]:
#Download study information
get_study(term, biome, lineage, file_study)

'01_Study_skin+Human+Host-associated_raw.csv'

In [118]:
#Load Study information
df1 = pd.read_csv(file_study)
#df1

In [119]:
#Select relevant information
df_study = df1[["Study ID","Study Name","Number Of Samples", "Submitted Date", "Experimental Factor", "Study Abstract"]]
#df_study

In [120]:
#Add information on biome
df2 = DataFrame(columns=("Biome","Lineage","Publication"))
#df2.index.name = 'No'

for i in tqdm_notebook(range(len(df_study))):
    with Session(API_BASE) as s:
        study = s.get('studies', df_study.loc[i, "Study ID"]).resource
        for biome in study.biomes:
            df2.loc[i] = [biome.biome_name,
                          biome.lineage,
                          study.publications
        ]
#df2

In [121]:
#Merge & Filter Table
df3 = pd.concat([df_study, df2], axis=1)
df3.set_index(["Study ID"])
df_study_biome = df3.query('Biome == ["Human", "Skin"] and Lineage == ["root:Host-associated:Human:Skin"]')
df_study_biome

,Study ID,Study Name,Number Of Samples,Submitted Date,Experimental Factor,Study Abstract,Biome,Lineage,Publication
1,SRP002480,Gene-Environment Interactions at the Skin Surface,2560,2016-02-03,null,16S rRNA gene sequences amplified from subject...,Skin,root:Host-associated:Human:Skin,[publications: 22310478 (2367526598248)]
2,ERP018577,Human skin bacterial and fungal microbiotas an...,96,2016-11-03,null,Using high-throughput 16S rDNA and ITS1 sequen...,Skin,root:Host-associated:Human:Skin,[]
3,ERP022958,Impact of the Mk VI SkinSuit on skin microbiot...,204,2017-06-16,null,Microgravity induces physiological decondition...,Skin,root:Host-associated:Human:Skin,[]
4,ERP019566,Longitudinal study of the diabetic skin and wo...,258,2017-11-27,null,Background: Type II diabetes is a chronic heal...,Skin,root:Host-associated:Human:Skin,[publications: 28740749 (2367526541632)]
5,ERP016629,Microbiome samples derived from Buruli ulcer w...,14,2016-07-29,null,Background: Buruli ulcer (BU) is an infectious...,Skin,root:Host-associated:Human:Skin,[publications: 28750103 (2367526498032)]
7,SRP056364,Skin microbiome in human volunteers inoculated...,191,2016-02-04,Infection,The aim of this project was to investigate the...,Skin,root:Host-associated:Human:Skin,[]


In [122]:
#Export study data
df_study_biome.to_csv('02_Study_Skin+Host-Associated+Human_filtered.csv')

## Choose relevant study
From the data above, we can filter which study can be used as the data for comparative microbiome analysis of human skin samples. Therefore we choose the study ID 'SRP002480' as our data

In [123]:
#Selected study
study = 'SRP002480'

### Get list of sample for a given study and its metadata

Get study: https://www.ebi.ac.uk/metagenomics/api/latest/studies/SRP002480
List samples: https://www.ebi.ac.uk/metagenomics/api/latest/studies/SRP002480/samples
Fetch samples for the given study accession: https://www.ebi.ac.uk/metagenomics/api/latest/samples?study_accession=SRP002480


In [124]:
#Fetch a list of sample data from a given study
filename_sample = '03_sample_'+study+'_raw.csv'
print(filename_sample)
if not os.path.isfile(filename_sample):
    with open(filename_sample, 'wb') as f:
        c = pycurl.Curl()
        c.setopt(c.URL, 'https://www.ebi.ac.uk/metagenomics/projects/'+study+'/overview/doExport')
        c.setopt(c.WRITEDATA, f)
        c.perform()
        c.close()

03_sample_SRP002480_raw.csv


In [92]:
#Filter relevant information from the list
df_sample = pd.read_csv(filename_sample)
df_sample_refine = df_sample[["Sample ID","Run ID","Release version"]]

In [105]:
#Create Container
if not os.path.isfile('04_raw_meta'+study+'.csv'):
    df_meta = DataFrame(columns=('Sex',"Body site"))
    df_meta.index.name = 'No'
else:
    df_meta = pd.read_csv('04_raw_meta_'+study+'.csv', index_col=0)

#Fetch metadata for given sample
pbar = tqdm_notebook(range(len(df_sample_refine))) #to make progressbar
for i in pbar:
    if not i in df_meta.index:
        with Session(API_BASE) as s:
            s_meta = s.get('samples', df_sample_refine.loc[i, "Sample ID"]).resource
            df_meta.loc[i] = [
                get_metadata(s_meta.sample_metadata, 'sex'),
                get_metadata(s_meta.sample_metadata, 'body site')
            ]
        pbar.set_description('processed: %d' % (i))
        pbar.update(1)
        sleep(1)

In [106]:
#Write to container
df_meta.to_csv('04_raw_meta_'+study+'.csv')
#df_meta

In [110]:
s_meta.sample_metadata

[{'key': 'geographic location (longitude)',
  'unit': None,
  'value': '38.984652 N 77.094709 W'},
 {'key': 'geographic location (country and/or sea,region)',
  'unit': None,
  'value': 'USA'},
 {'key': 'collection date', 'unit': None, 'value': '2013-05-22'},
 {'key': 'environment (biome)', 'unit': None, 'value': 'human skin'},
 {'key': 'environment (feature)', 'unit': None, 'value': 'Vf-R'},
 {'key': 'NCBI sample classification', 'unit': None, 'value': '539655'},
 {'key': 'instrument model', 'unit': None, 'value': 'Illumina HiSeq 2000'},
 {'key': 'host', 'unit': None, 'value': 'human'}]

In [108]:
#Merge metadata with the raw sample list
result = pd.concat([df_sample_refine, df_meta], axis=1)
result.to_csv('05_sample_'+study+'_meta.csv')

### Query samples based on its metadata

In [109]:
#Load sample list
df_result_raw = pd.read_csv('05_sample_'+study+'_meta.csv', index_col = 0)
df_result_raw

,Sample ID,Run ID,Release version,Sex,Body site
0,SRS451417,SRR919527,2.0,male,antecubital crease
1,SRS451417,SRR919587,2.0,male,antecubital crease
2,SRS451418,SRR919528,2.0,male,back
3,SRS451418,SRR919588,2.0,male,back
4,SRS451419,SRR919529,2.0,male,external auditory canal
5,SRS451419,SRR919589,2.0,male,external auditory canal
6,SRS451420,SRR919530,2.0,male,hypothenar palm
7,SRS451420,SRR919590,2.0,male,hypothenar palm
8,SRS451421,SRR919531,2.0,male,retroauricular crease
9,SRS451421,SRR919591,2.0,male,retroauricular crease


In [111]:
df_result_raw.columns = df_result_raw.columns.str.replace(' ', '_') #Refine the data to make it easier for filtering 

#Query for back sample from male population
df_result_m_back = df_result_raw.query('Sex == ["male "] and Body_site == ["back "]')
df_result_m_back

,Sample_ID,Run_ID,Release_version,Sex,Body_site
2,SRS451418,SRR919528,2.0,male,back
3,SRS451418,SRR919588,2.0,male,back
20,SRS451427,SRR919536,2.0,male,back
21,SRS451427,SRR919596,2.0,male,back
42,SRS451438,SRR919548,2.0,male,back
43,SRS451438,SRR919608,2.0,male,back
190,SRS451613,SRR919884,2.0,male,back
191,SRS451613,SRR919934,2.0,male,back
226,SRS451712,SRR920114,2.0,male,back
227,SRS451712,SRR920161,2.0,male,back


## Random sampling

In [185]:
df = {'ID': ['H900','H901','H902','','M1435','M149','M157','','M699','M920','','M789','M617','M991','H903','M730','M191'],
  'Clone': [0,1,2,2,2,2,2,2,3,3,3,4,4,4,5,5,6],
  'Length': [48,42  ,48,48,48,48,48,48,48,48,48,48,48,48,48,48,48]}

df = pd.DataFrame(df)
print (df)
df = df[df.ID != '']

rows = np.random.choice(df.index.values, 5)
sampled_df = df.ix[rows]
print (sampled_df)

    Clone     ID  Length
0       0   H900      48
1       1   H901      42
2       2   H902      48
3       2             48
4       2  M1435      48
5       2   M149      48
6       2   M157      48
7       2             48
8       3   M699      48
9       3   M920      48
10      3             48
11      4   M789      48
12      4   M617      48
13      4   M991      48
14      5   H903      48
15      5   M730      48
16      6   M191      48


C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:10: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate_ix
  # Remove the CWD from sys.path while we load stuff.


    Clone    ID  Length
16      6  M191      48
16      6  M191      48
8       3  M699      48
13      4  M991      48
1       1  H901      42


### Get analysis result of a given sample in a study
https://www.ebi.ac.uk/metagenomics//projects/SRP002480/samples/SRS451457/runs/SRR919567/results/versions/2.0/taxonomy/OTU-table-HDF5-BIOM

In [125]:
#Create ouput folder
cwd = os.getcwd() #get current working directory
output_folder = "\output_"+study #name of the output folder for a given study, use \ for directory in windows
if not os.path.isdir(cwd + output_folder):
    os.mkdir(cwd + output_folder)
new_dir = cwd + output_folder 
new_dir

'E:\\Jupyter_Lab\\KetiakProject\\src\\output_SRP002480'

In [183]:
run = 'SRR919528'
extension = 'JSON Biom'

#Ambil data dari EBI
for i in tqdm_notebook(df_result_m_back.index):
    os.chdir(new_dir) #pindah ke folder output
    filename = df_result_m_back.loc[i, "Sample_ID"]+\
    '_'+df_result_m_back.loc[i, "Run_ID"]+\
    '_'+df_result_m_back.loc[i, "Sex"]+\
    df_result_m_back.loc[i, "Body_site"]+\
    '.biom'
    if not os.path.isfile(filename):
        with open(filename, 'wb') as f:
            c = pycurl.Curl()
            c.setopt(c.URL, get_analysis_result(run, extension))
            c.setopt(c.WRITEDATA, f)
            c.perform()
            c.close()
    os.chdir(cwd) #balik ke folder semula

### References
---
<a id='ref1'></a>
1. Alex L Mitchell, Maxim Scheremetjew, Hubert Denise, Simon Potter, Aleksandra Tarkowska, Matloob Qureshi, Gustavo A Salazar, Sebastien Pesseat, Miguel A Boland, Fiona M I Hunter, Petra ten Hoopen, Blaise Alako, Clara Amid, Darren J Wilkinson, Thomas P Curtis, Guy Cochrane, Robert D Finn; EBI Metagenomics in 2017: enriching the analysis of microbial communities, from sequence reads to assemblies, Nucleic Acids Research, Volume 46, Issue D1, 4 January 2018, Pages D726–D735, https://doi.org/10.1093/nar/gkx967